In [1]:
using LinearAlgebra
using BenchmarkTools
using PyPlot
using FFTW

In [40]:
x = rand(1024)
fft([1:16...])

16-element Vector{ComplexF64}:
 136.0 + 0.0im
  -8.0 + 40.218715937006785im
  -8.0 + 19.31370849898476im
  -8.0 + 11.972846101323912im
  -8.0 + 8.0im
  -8.0 + 5.345429103354389im
  -8.0 + 3.313708498984761im
  -8.0 + 1.5912989390372658im
  -8.0 + 0.0im
  -8.0 - 1.5912989390372658im
  -8.0 - 3.313708498984761im
  -8.0 - 5.345429103354389im
  -8.0 - 8.0im
  -8.0 - 11.972846101323912im
  -8.0 - 19.31370849898476im
  -8.0 - 40.218715937006785im

In [64]:
function cooley_tukey(x)
    N = length(x)

    if (N > 2)
        x_odd = cooley_tukey(x[1:2:N])
        x_even = cooley_tukey(x[2:2:N])
    else
        x_odd = x[1]
        x_even = x[2]
    end
    n = 0:length(x_odd)-1
    half = div(N,2)
    println(n)
    factor = exp.(-2im*pi*n/N)
    println(factor)
    println( vcat(x_odd .+ x_even .* factor[1:half],
                x_odd .- x_even .* factor[1:half]))
    return vcat(x_odd .+ x_even .* factor[1:half],
                x_odd .- x_even .* factor[1:half])

end

cooley_tukey (generic function with 1 method)

In [65]:
cooley_tukey([1:8...])

0:0
ComplexF64[1.0 - 0.0im]
ComplexF64[6.0 - 0.0im, -4.0 + 0.0im]
0:0
ComplexF64[1.0 - 0.0im]
ComplexF64[10.0 - 0.0im, -4.0 + 0.0im]
0:1
ComplexF64[1.0 - 0.0im, 6.123233995736766e-17 - 1.0im]
ComplexF64[16.0 - 0.0im, -4.0 + 4.0im, -4.0 + 0.0im, -3.9999999999999996 - 4.0im]
0:0
ComplexF64[1.0 - 0.0im]
ComplexF64[8.0 - 0.0im, -4.0 + 0.0im]
0:0
ComplexF64[1.0 - 0.0im]
ComplexF64[12.0 - 0.0im, -4.0 + 0.0im]
0:1
ComplexF64[1.0 - 0.0im, 6.123233995736766e-17 - 1.0im]
ComplexF64[20.0 - 0.0im, -4.0 + 4.0im, -4.0 + 0.0im, -3.9999999999999996 - 4.0im]
0:3
ComplexF64[1.0 - 0.0im, 0.7071067811865476 - 0.7071067811865475im, 6.123233995736766e-17 - 1.0im, -0.7071067811865475 - 0.7071067811865476im]
ComplexF64[36.0 - 0.0im, -4.0 + 9.65685424949238im, -4.0 + 4.0im, -4.0 + 1.6568542494923797im, -4.0 + 0.0im, -3.9999999999999996 - 1.6568542494923797im, -3.9999999999999996 - 4.0im, -3.9999999999999987 - 9.65685424949238im]


8-element Vector{ComplexF64}:
                36.0 - 0.0im
                -4.0 + 9.65685424949238im
                -4.0 + 4.0im
                -4.0 + 1.6568542494923797im
                -4.0 + 0.0im
 -3.9999999999999996 - 1.6568542494923797im
 -3.9999999999999996 - 4.0im
 -3.9999999999999987 - 9.65685424949238im

In [9]:
cooley_tukey(x)

  2.745 ms (16368 allocations: 1.41 MiB)


1024-element Vector{ComplexF64}:
   513.8909618317466 - 0.0im
   5.761886723876073 + 3.4270554651149783im
  1.5693308225555063 + 5.379970864297059im
  -5.590677421672849 - 3.6441369650192676im
   5.771590179110686 - 8.916034114648337im
    8.06425919108073 - 8.642533499054089im
   1.753568893150483 + 21.54031665861823im
   6.070425148858878 - 8.932239373030464im
 0.22171019654952856 - 3.4142620880842864im
 -10.656919293429123 - 9.821826582312426im
  0.7131246272447607 + 6.636009638382709im
  -3.783540154381813 - 3.138217968366283im
 -2.9117044909899636 - 6.5965407360321064im
                     ⋮
 -2.9117044909899636 + 6.596540736032105im
  -3.783540154381818 + 3.1382179683662805im
  0.7131246272447656 - 6.636009638382711im
 -10.656919293429134 + 9.82182658231242im
 0.22171019654952373 + 3.4142620880842838im
   6.070425148858873 + 8.932239373030468im
  1.7535688931504891 - 21.54031665861823im
   8.064259191080723 + 8.642533499054096im
    5.77159017911068 + 8.916034114648342im
  -5.59

In [18]:
function fft1d(x, isign)
    N = length(x)

    # swap elements in array
    j = 1
    for i = 1:N
        if j > i
            temp = x[i]
            x[i] = x[j]
            x[j] = temp
        end
        m = N >> 1
        while m >= 1 && j > m
            j -= m
            m >>= 1
        end
        j += m
    end

    step = 2
    while step <= N
        half_step = step ÷ 2
        w_m = exp(-2im* isign * π / step)

        for k in 1:step:N
            w = 1.0
            for m in 0:(half_step-1)
                u = x[k + m]
                t = w * x[k + m + half_step]
                x[k + m] = u + t
                x[k + m + half_step] = u - t
                w *= w_m
            end
        end
        step *= 2
    end

    if isign == -1
        for i in 1:N
            x[i] /= N
        end
    end
    
    return x
end

fft1d (generic function with 1 method)

In [22]:
x = convert(Array{ComplexF64, 1}, [1:8...])
fft1d(fft1d(x, 1), -1)

8-element Vector{ComplexF64}:
                1.0 + 0.0im
 2.0000000000000004 - 2.718345674301793e-16im
                3.0 + 4.440892098500626e-16im
                4.0 + 3.8285686989269494e-16im
                5.0 + 0.0im
                6.0 - 4.978996250514798e-17im
                7.0 - 4.440892098500626e-16im
                8.0 - 6.123233995736767e-17im

In [23]:
x = convert(Array{ComplexF64, 1}, [1:8...])
ifft(fft(x))

8-element Vector{ComplexF64}:
 1.0 + 0.0im
 2.0 + 0.0im
 3.0 + 0.0im
 4.0 + 0.0im
 5.0 + 0.0im
 6.0 + 0.0im
 7.0 + 0.0im
 8.0 + 0.0im

In [57]:
function bit_reversal_permutation!(x, dim)
    N = size(x, dim)
    j = 1
    for i in 1:N
        if j > i
            if dim == 1
                temp       = x[i, :, :]
                x[i, :, :] = x[j, :, :]
                x[j, :, :] = temp
            elseif dim == 2
                temp       = x[:, i, :]
                x[:, i, :] = x[:, j, :]
                x[:, j, :] = temp
            elseif dim == 3
                temp       = x[:, :, i]
                x[:, :, i] = x[:, :, j]
                x[:, :, j] = temp
            end
        end
        m = N >> 1
        while m >= 1 && j > m
            j -= m
            m >>= 1
        end
        j += m
    end
end

function fft1d_dim!(x, dim, isign)
    N = size(x, dim)

    step = 2
    while step <= N
        half_step = step ÷ 2
        w_m = exp(-2im* isign * π / step)

        if dim == 1
            for k in 1:step:N, l in 1:size(x, 2), m in 1:size(x, 3)
                w = 1.0
                for n in 0:(half_step-1)
                    u = x[k + n, l, m]
                    t = w * x[k + n + half_step, l, m]
                    x[k + n, l, m] = u + t
                    x[k + n + half_step, l, m] = u - t
                    w *= w_m
                end
            end
        elseif dim == 2
            for k in 1:size(x, 1), l in 1:step:N, m in 1:size(x, 3)
                w = 1.0
                for n in 0:(half_step-1)
                    u = x[k, l + n, m]
                    t = w * x[k, l + n + half_step, m]
                    x[k, l + n, m] = u + t
                    x[k, l + n + half_step, m] = u - t
                    w *= w_m
                end
            end
        elseif dim == 3
            for k in 1:size(x, 1), l in 1:size(x, 2), m in 1:step:N
                w = 1.0
                for n in 0:(half_step-1)
                    u = x[k, l, m + n]
                    t = w * x[k, l, m + n + half_step]
                    x[k, l, m + n] = u + t
                    x[k, l, m + n + half_step] = u - t
                    w *= w_m
                end
            end
        end

        step *= 2
    end

    if isign == -1
        x .= x ./ N
    end
    
    return x
end

function fft3d(x, isign)
    for dim in 1:3
        bit_reversal_permutation!(x, dim)
        fft1d_dim!(x, dim, isign)
    end
    return x
end

fft3d (generic function with 1 method)

In [222]:
nx = 16
x = convert(Array{ComplexF64, 3}, rand(nx,nx,nx));
temp = similar(x)
temp .= x;

In [223]:
fft(x)

16×16×16 Array{ComplexF64, 3}:
[:, :, 1] =
    2047.2+0.0im       16.0887+10.1869im   …   16.0887-10.1869im
   24.3233+21.7546im   -7.2425+16.4597im      -7.52384-0.257674im
 -0.655054-1.42161im   -15.952+37.9304im      -9.31828+6.05375im
  -3.85718+5.08988im  -12.1058-8.67421im       25.2519+1.9285im
   1.53521-8.66543im   13.8437-1.46624im       1.08306-14.2283im
  -5.34719+2.01609im   22.0902+12.0757im   …  -13.5612-21.502im
  0.518549-4.30632im   1.68197+3.51058im      -22.1339+13.5552im
  -1.38445+20.5723im   8.59831+13.6736im       5.32042+3.8039im
   7.98016+0.0im      -2.51477-6.12254im      -2.51477+6.12254im
  -1.38445-20.5723im   5.32042-3.8039im        8.59831-13.6736im
  0.518549+4.30632im  -22.1339-13.5552im   …   1.68197-3.51058im
  -5.34719-2.01609im  -13.5612+21.502im        22.0902-12.0757im
   1.53521+8.66543im   1.08306+14.2283im       13.8437+1.46624im
  -3.85718-5.08988im   25.2519-1.9285im       -12.1058+8.67421im
 -0.655054+1.42161im  -9.31828-6.05375im       -1

In [224]:
fft3d(x, 1)

16×16×16 Array{ComplexF64, 3}:
[:, :, 1] =
    2047.2+0.0im       16.0887+10.1869im   …   16.0887-10.1869im
   24.3233+21.7546im   -7.2425+16.4597im      -7.52384-0.257674im
 -0.655054-1.42161im   -15.952+37.9304im      -9.31828+6.05375im
  -3.85718+5.08988im  -12.1058-8.67421im       25.2519+1.9285im
   1.53521-8.66543im   13.8437-1.46624im       1.08306-14.2283im
  -5.34719+2.01609im   22.0902+12.0757im   …  -13.5612-21.502im
  0.518549-4.30632im   1.68197+3.51058im      -22.1339+13.5552im
  -1.38445+20.5723im   8.59831+13.6736im       5.32042+3.8039im
   7.98016+0.0im      -2.51477-6.12254im      -2.51477+6.12254im
  -1.38445-20.5723im   5.32042-3.8039im        8.59831-13.6736im
  0.518549+4.30632im  -22.1339-13.5552im   …   1.68197-3.51058im
  -5.34719-2.01609im  -13.5612+21.502im        22.0902-12.0757im
   1.53521+8.66543im   1.08306+14.2283im       13.8437+1.46624im
  -3.85718-5.08988im   25.2519-1.9285im       -12.1058+8.67421im
 -0.655054+1.42161im  -9.31828-6.05375im       -1